In [1]:
%load_ext autoreload
%autoreload 2
%cd ../..

import pickle
import numpy as np
import pandas as pd
from evoaug_tf import evoaug, augment
from src.diff_expression_model import get_model, get_siamese_model, post_hoc_conjoining, get_auroc

/home/ubuntu/DeepDifE


2024-08-09 11:19:25.461706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 11:19:25.504893: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 11:19:25.506005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 11:19:26.226815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Prepare the data

For now we start from this data pickle as I'm not aware how Helder did DE analysis and generated the labels

In [2]:
ppath = "data/dataset_solid_chrome.pkl"
with open(ppath, 'rb') as f:
    data = pickle.load(f)

In [3]:
data

,Category,GeneFamily,seqs,ohs,rcohs,ohsDuo,in_original_balanced,set,npshap-single,npshap-posthoc
geneID,,,,,,,,,,
AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1...",True,train,"[[[[-0.00029412], [0.00011191], [0.00071743], ...","[[[[-0.00024759], [0.00013361], [0.00015087], ..."
AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1...",True,valid,"[[[[0.00049611], [0.00069235], [0.00377408], [...","[[[[0.00051932], [0.00032187], [0.00027891], [..."
AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0...",True,valid,"[[[[-0.00030243], [-1.08283739e-05], [0.000359...","[[[[-0.00031496], [-8.34961975e-05], [0.000132..."
AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...","[[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0...",True,train,"[[[[0.00254294], [0.00639127], [0.00152307], [...","[[[[0.00208253], [0.0018626], [0.00023493], [0..."
AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,...","[[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1...",True,valid,"[[[[-5.23597009e-05], [0.00045373], [0.0003575...","[[[[0.00051831], [0.00050734], [4.30682958e-05..."
...,...,...,...,...,...,...,...,...,...,...
AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1...",True,test,"[[[[-1.85048248e-05], [5.31432997e-05], [0.000...","[[[[0.00034249], [0.00039158], [1.82568183e-05..."
AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0...",True,train,"[[[[9.53665585e-05], [0.00032812], [0.00046541...","[[[[-5.87131144e-05], [6.68894427e-05], [6.172..."
AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0...",True,test,"[[[[0.00061825], [6.60565856e-05], [5.89712399...","[[[[0.00044855], [6.3632986e-05], [0.00018469]..."


In [4]:
data["set"].value_counts()

train    1900
valid     257
test      241
Name: set, dtype: int64

To show how to subdivide the dataset into train-test split we only take the following columns

In [5]:
dataset = data.reset_index()
dataset = dataset[["geneID", "Category", "GeneFamily", "seqs"]]
dataset.rename(columns={"geneID":"GeneID", "Category":"Label", "seqs": "Sequence"}, inplace=True)

## Phylo augmentation 

Define the fasta files for each species of which we want to use orthologs.

In [6]:
phylo_augmentation_base_path = "data/phylo_aug/fasta/"
species_fasta_files = {
			"aly":"aly.tss.fasta",
			"chi":"chi.tss.fasta",
			"cru":"cru.tss.fasta",
			"aar":"aar.tss.fasta",
			"ath-an1":"ath-an1.tss.fasta",
			"ath-c24":"ath-c24.tss.fasta",
			"ath-cvi":"ath-cvi.tss.fasta",
			"ath-eri":"ath-eri.tss.fasta",
			"ath-kyo":"ath-kyo.tss.fasta",
			"ath-ler":"ath-ler.tss.fasta",
			"ath-sha":"ath-sha.tss.fasta",
			"ath":"ath.tss.fasta",
			"chi":"chi.tss.fasta",
			"cpa":"cpa.tss.fasta",
			"cru":"cru.tss.fasta",
			"esa":"esa.tss.fasta",
			"spa":"spa.tss.fasta",
			"tha":"tha.tss.fasta"
}

species_to_include = ["aly","chi","cru"]

Parsing the fasta files. In this case the label of the original sequence is incorporated in the ids in the fasta file
e.g:
```
>Aa31LG6G5540_1
TTGTATAAATATATTAAATT
```

In [7]:
from Bio import SeqIO

sequence_list = []
gene_id_list = []
label_list = []
species_list = []
for species in species_to_include:
	species_fasta = species_fasta_files[species]
	fasta_path = f"{phylo_augmentation_base_path}{species_fasta}"
	fasta_sequences = SeqIO.parse(open(fasta_path),'fasta')
	print(f"File: {fasta_path} done!")
	for fasta in fasta_sequences:
		name, sequence = fasta.id, str(fasta.seq)
		
		# The label is stored inside the id name (e.g. >ATERI-4G42060_1), split these
		gene_id = name.split("_")[0]
		label = name.split("_")[1]
		species = species_fasta.split(".")[0]

		gene_id_list.append(gene_id)
		sequence_list.append(sequence)
		label_list.append(label)
		species_list.append(species)

File: data/phylo_aug/fasta/aly.tss.fasta done!
File: data/phylo_aug/fasta/chi.tss.fasta done!
File: data/phylo_aug/fasta/cru.tss.fasta done!


In [8]:
phylo_aug_df = pd.DataFrame({"GeneID": gene_id_list, "Label": label_list, "Sequence": sequence_list, "Species": species_list})

In [9]:
phylo_aug_df

,GeneID,Label,Sequence,Species
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly
...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru


### Get gene family of the original ortholog

Because during the batch time augmentation a original sample will be swapped by one of its orthologs, we need to know from which original sample each ortholog originates from.

In [10]:
base_dir_ortholog = "data/phylo_aug/orthologs/"
ortholog_files = {
			"ath-aly":"ath_aly.txt",
			"ath-chi":"ath_chi.txt",
			"ath-cru":"ath_cru.txt",
			"ath-aar":"ath_aar.txt",
			"ath-aly":"ath_aly.txt",
			"ath-ath-an1":"ath_ath-an1.txt",
			"ath-ath-c24":"ath_ath-c24.txt",
			"ath-ath-cvi":"ath_ath-cvi.txt",
			"ath-ath-eri":"ath_ath-eri.txt",
			"ath-ath-kyo":"ath_ath-kyo.txt",
			"ath-ath-ler":"ath_ath-ler.txt",
			"ath-ath-sha":"ath_ath-sha.txt",
			"ath-ath":"ath_ath.txt",
			"ath-chi":"ath_chi.txt",
			"ath-cpa":"ath_cpa.txt",
			"ath-cru":"ath_cru.txt",
			"ath-esa":"ath_esa.txt",
			"ath-spa":"ath_spa.txt",
			"ath-tha":"ath_tha.txt"
			}

Create a lookup table to include the orthologs in the dataframe

In [11]:
lookup_table = {}
for orthologs in ortholog_files.values():
	with open(f"{base_dir_ortholog}{orthologs}", 'r') as file:
		for line in file:
			# Split each line by tab
			columns = line.strip().split('\t')
			if len(columns) >= 2:  # Ensure there are at least two columns
				key = columns[1]
				value = columns[0]
				lookup_table[key] = value

In [12]:
phylo_aug_df["Ortholog"] = phylo_aug_df["GeneID"].map(lookup_table)

In [13]:
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260
...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530


In [14]:
phylo_aug_df = pd.merge(phylo_aug_df, dataset[["GeneID", "GeneFamily"]], left_on="Ortholog", right_on="GeneID", how="left")

In [15]:
phylo_aug_df.drop("GeneID_y", axis=1, inplace=True)
phylo_aug_df.rename(columns={"GeneID_x": "GeneID"}, inplace=True)
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130
...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279


### One-hot-encode & reverse-complement both dataset as augmented dataset

In [16]:
from src.prepare_dataset import one_hot_encode_series, reverse_complement_series, reverse_complement_sequence
dataset["One_hot_encoded"] = one_hot_encode_series(dataset["Sequence"])
phylo_aug_df["One_hot_encoded"] = one_hot_encode_series(phylo_aug_df["Sequence"])

In [17]:
dataset["RC_one_hot_encoded"] = reverse_complement_series(dataset["One_hot_encoded"])
phylo_aug_df["RC_one_hot_encoded"] = reverse_complement_series(phylo_aug_df["One_hot_encoded"])

In [18]:
dataset

,GeneID,Label,GeneFamily,Sequence,One_hot_encoded,RC_one_hot_encoded
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,..."
...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,..."


In [19]:
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130,"[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1,..."
...,...,...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357,"[[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


In [20]:
phylo_aug_df = phylo_aug_df[phylo_aug_df["GeneFamily"].notna()]
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130,"[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1,..."
...,...,...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357,"[[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


### Train-test split

In [21]:
from src.prepare_dataset import grouped_shuffle_split
train_df, validation_test_df = grouped_shuffle_split(dataset, dataset["GeneFamily"], 0.2)

In [22]:
validation_df, test_df  = grouped_shuffle_split(validation_test_df, validation_test_df["GeneFamily"], 0.5)

In [23]:
train_df = train_df.sample(700, random_state=5)

In [24]:
print(f"Length of training set: {train_df.shape[0]}")
print(f"Length of validation set: {validation_df.shape[0]}")
print(f"Length of test set: {test_df.shape[0]}")

Length of training set: 700
Length of validation set: 257
Length of test set: 241


### Filter out the orthologs from samples in test or validation

In [25]:
filtered_phylo_aug_df = phylo_aug_df[~phylo_aug_df["Ortholog"].isin(validation_test_df["GeneID"])]

In [26]:
filtered_phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
5,AL6G25970,0,GATTGAGAGTGGGTGTCATAGTTTCTTTTAATTTTTTTGCTTATCT...,aly,AT5G15150,HOM04D000209,"[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
...,...,...,...,...,...,...,...,...
2787,Carub.0005s1225,1,GAAAGAATGGAGAGAGAAGAGAGACGAGTTCAAGAAGAAAGTGAGC...,cru,AT3G46450,HOM04D003906,"[[0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...","[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."
2788,Carub.0005s2556,1,TTGTGTAGTGCCTTGAAAACGTCCTGATCCCTCCAAAGATGTTACA...,cru,AT3G58780,HOM04D000037,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


## Batch time augmentation 

In [27]:
import random

def get_batch(data, phylo_aug_data, indices, perform_phylo_aug=True, phylo_aug_rate=1.0):
	"""
	Creates a batch of the input and one-hot encodes the sequences
	"""
	seqs = []
	labels = []
	gene_ids = []
	for ii in indices:
		gene_id = data.iloc[ii].GeneID
		sample = data.iloc[ii]
		if random.choices(population=[0, 1], weights=[1 - phylo_aug_rate, phylo_aug_rate])[0] == 1:
			orthologs = phylo_aug_data[phylo_aug_data.Ortholog == gene_id]
			if not orthologs.empty:
				sample = phylo_aug_data[phylo_aug_data.Ortholog == gene_id].sample().iloc[0]
			# Randomly pick the forward or reverse complement strand
		if random.randint(0, 1) == 0:
			gene_ids.append(sample.GeneID)
			seqs.append(sample.RC_one_hot_encoded)
		else:
			gene_ids.append(sample.GeneID)
			seqs.append(sample.One_hot_encoded)
			
		labels.append(sample.Label)

	X_batch = np.array(seqs)
	X_batch = X_batch.astype('float32')

	Y_batch = np.array(labels)
	Y_batch = Y_batch.astype('int64')

	return X_batch, Y_batch

In [28]:
def data_gen(data, phylo_aug_data, batch_size, perform_phylo_aug=True, phylo_aug_rate=1.0):
	"""
	Generator function for loading input data in batches
	"""
	num_samples = data.shape[0]
	# indices = list(range(num_samples))
	indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)
	# indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)

	ii = 0
	while True:
		yield get_batch(data, phylo_aug_data, indices[ii:ii + batch_size], perform_phylo_aug, phylo_aug_rate)
		ii += batch_size
		if ii >= num_samples:
			ii = 0
			indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)
			# indices = list(range(num_samples))


## Prepare validation input

In [29]:
def get_input_and_labels(df):
	ohe_np = np.stack(df["One_hot_encoded"])
	rc_np = np.stack(df["RC_one_hot_encoded"])

	x = np.append(ohe_np, rc_np, axis=0)
	x = x.astype('float32')
	y = np.append(df["Label"], df["Label"])
	y = y.astype('int64')
	return x, y

In [30]:
x_validation, y_validation = get_input_and_labels(validation_df)

As we do batch time augmentation we define the data generator here

In [31]:
BATCH_SIZE = 100
train_data_gen = data_gen(train_df,filtered_phylo_aug_df, BATCH_SIZE, True, 0.6)

## Prepare model

Get the shape of the input data

In [32]:
input_shape = train_df["One_hot_encoded"].iloc[0].shape

Initialize the model

In [34]:
model = get_model(input_shape=input_shape, perform_evoaug=False,learning_rate=0.001)

## Train the model

In [35]:
# early stopping callback
import tensorflow as tf

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=40,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# reduce learning rate callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
												factor=0.1,
												patience=5,
												min_lr=1e-7,
												mode='min',
												verbose=1)
callbacks = [early_stopping_callback, reduce_lr]

In [36]:
import math
history = model.fit(train_data_gen,
					steps_per_epoch=math.ceil(
						train_df.shape[0] / BATCH_SIZE),
					epochs=100,
					validation_data=(x_validation, y_validation),
					callbacks=callbacks
					)

Epoch 1/100
7/7 [==============================] - 3s 208ms/step - loss: 0.6995 - acc: 0.4643 - auROC: 0.4453 - auPRC: 0.4597 - true_positives_1: 274.0000 - val_loss: 0.6937 - val_acc: 0.4961 - val_auROC: 0.4935 - val_auPRC: 0.4895 - val_true_positives_1: 247.0000 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 1s 104ms/step - loss: 0.6954 - acc: 0.4800 - auROC: 0.4862 - auPRC: 0.4895 - true_positives_1: 66.0000 - val_loss: 0.6946 - val_acc: 0.4981 - val_auROC: 0.5046 - val_auPRC: 0.5042 - val_true_positives_1: 0.0000e+00 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 1s 105ms/step - loss: 0.6945 - acc: 0.4971 - auROC: 0.5066 - auPRC: 0.5060 - true_positives_1: 27.0000 - val_loss: 0.6936 - val_acc: 0.4864 - val_auROC: 0.4841 - val_auPRC: 0.4889 - val_true_positives_1: 29.0000 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 1s 102ms/step - loss: 0.6939 - acc: 0.5071 - auROC: 0.4876 - auPRC: 0.4853 - true_positives_1: 239.0000 - val_loss: 

In [37]:
auroc_values = history.history['val_loss']

# Find the index of the epoch where AUROC is the maximum
max_auroc_epoch = np.argmin(auroc_values)

# Extract all the metrics for the epoch with the maximum AUROC
metrics_at_max_auroc = {metric: values[max_auroc_epoch] for metric, values in history.history.items()}
metrics_at_max_auroc


{'loss': 0.6945062875747681,
 'acc': 0.49714285135269165,
 'auROC': 0.5066490173339844,
 'auPRC': 0.5060290098190308,
 'true_positives_1': 27.0,
 'val_loss': 0.6936250925064087,
 'val_acc': 0.48638132214546204,
 'val_auROC': 0.4841478765010834,
 'val_auPRC': 0.48889878392219543,
 'val_true_positives_1': 29.0,
 'lr': 0.001}

### Finetune phylo-aug

In [38]:
x_train_pure, y_train_pure = get_input_and_labels(train_df)


In [39]:
from tensorflow import keras
from src.diff_expression_model import compile_model

# set up callbacks
model = compile_model(model, learning_rate=0.0001)

# train model
finetune_history = model.fit(x_train_pure, y_train_pure,
						epochs=10,
						batch_size=100,
						shuffle=True,
						validation_data=(x_validation, y_validation),
						callbacks=[early_stopping_callback])

Epoch 1/10
14/14 [==============================] - 2s 40ms/step - loss: 0.6491 - acc: 0.6329 - auROC: 0.6704 - auPRC: 0.6425 - true_positives_1: 454.0000 - val_loss: 0.6621 - val_acc: 0.6051 - val_auROC: 0.6491 - val_auPRC: 0.6293 - val_true_positives_1: 202.0000
Epoch 2/10
14/14 [==============================] - 0s 19ms/step - loss: 0.6437 - acc: 0.6343 - auROC: 0.6805 - auPRC: 0.6409 - true_positives_1: 482.0000 - val_loss: 0.6576 - val_acc: 0.6109 - val_auROC: 0.6508 - val_auPRC: 0.6300 - val_true_positives_1: 165.0000
Epoch 3/10
14/14 [==============================] - 0s 19ms/step - loss: 0.6294 - acc: 0.6514 - auROC: 0.7026 - auPRC: 0.6764 - true_positives_1: 458.0000 - val_loss: 0.6587 - val_acc: 0.6128 - val_auROC: 0.6503 - val_auPRC: 0.6297 - val_true_positives_1: 185.0000
Epoch 4/10
14/14 [==============================] - 0s 18ms/step - loss: 0.6442 - acc: 0.6386 - auROC: 0.6813 - auPRC: 0.6670 - true_positives_1: 486.0000 - val_loss: 0.6591 - val_acc: 0.6051 - val_auROC: 

In [40]:
auroc_values = finetune_history.history['val_loss']

# Find the index of the epoch where AUROC is the maximum
max_auroc_epoch = np.argmin(auroc_values)

# Extract all the metrics for the epoch with the maximum AUROC
metrics_at_max_auroc = {metric: values[max_auroc_epoch] for metric, values in finetune_history.history.items()}
metrics_at_max_auroc


{'loss': 0.6436826586723328,
 'acc': 0.6342856884002686,
 'auROC': 0.6804790496826172,
 'auPRC': 0.6408889889717102,
 'true_positives_1': 482.0,
 'val_loss': 0.6576040387153625,
 'val_acc': 0.6108949184417725,
 'val_auROC': 0.6508070230484009,
 'val_auPRC': 0.6299607157707214,
 'val_true_positives_1': 165.0}

## Test set

We perform post-hoc conjoing

In [41]:
siamese_model = get_siamese_model(model)

Prepare the test set data

In [42]:
x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

In [43]:
y_test = test_df["Label"].to_numpy()

As we used evo aug in our model all the sequences which were trained on, were of length 620. We use the evoaug padding function to pad the test set to the same length

In [44]:
x_test = model._pad_end(x_test)
x_test_rc = model._pad_end(x_test_rc)

AttributeError: 'Sequential' object has no attribute '_pad_end'

In [45]:
predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

8/8 [==============================] - 0s 4ms/step


In [46]:
get_auroc(y_test, predictions)

0.6568953568953568

In [36]:
a=1

In [115]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_categories))

              precision    recall  f1-score   support

           0       0.64      0.64      0.64       111
           1       0.69      0.69      0.69       130

    accuracy                           0.67       241
   macro avg       0.67      0.67      0.67       241
weighted avg       0.67      0.67      0.67       241

